In [182]:
import numpy as np
import pandas as pd
import requests
import io
import plotly.express as px
import qgrid
import plotly.graph_objects as go

In [183]:
RECOVERED='Recovered'
DEATH='Deaths'
CONFIRMED='Confirmed'
INFECTED='Infected'

In [184]:
confirmed_url='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
confirmed_data=pd.read_csv(confirmed_url)
confirmed_data.to_csv('confirmed_data.csv')

death_url='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
death_data=pd.read_csv(death_url)
death_data.to_csv('death_data.csv')

recovered_url='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
recovered_data=pd.read_csv(recovered_url)
recovered_data.to_csv('recovered_data.csv')

LAST_COLUMN=confirmed_data.columns[-1]

# confirmed_data.info()

In [185]:
def extractDataFromCountry(dataset,country):
    country_data = dataset[dataset['Country/Region']==country]
    return country_data

In [186]:
confirm_dataset=pd.DataFrame(confirmed_data,columns=['Province/State','Country/Region','Lat','Long',LAST_COLUMN]).rename(columns={LAST_COLUMN:CONFIRMED})
death_dataset=pd.DataFrame(death_data,columns=['Province/State','Country/Region','Lat','Long',LAST_COLUMN]).rename(columns={LAST_COLUMN:DEATH})
recover_dataset=pd.DataFrame(recovered_data,columns=['Province/State','Country/Region','Lat','Long',LAST_COLUMN]).rename(columns={LAST_COLUMN:RECOVERED})
merged_dataset=confirm_dataset.merge(death_dataset,on=['Province/State','Country/Region','Lat','Long']).merge(recover_dataset,on=['Province/State','Country/Region','Lat','Long'])

merged_dataset[INFECTED] = merged_dataset[CONFIRMED] - merged_dataset[DEATH] -merged_dataset[RECOVERED]
merged_dataset['Country/Region'] = merged_dataset['Country/Region'].replace('Mainland China', 'China')
merged_dataset[['Province/State']] = merged_dataset[['Province/State']].fillna('NA')
merged_dataset[[CONFIRMED, DEATH, RECOVERED, INFECTED]] = merged_dataset[[CONFIRMED, DEATH, RECOVERED, INFECTED]].fillna(0)


In [187]:
merged_dataset_latest = merged_dataset.groupby('Country/Region').sum().reset_index()
# merged_dataset_latest=merged_dataset[merged_dataset['Country/Region']=='Canada']
merged_dataset_latest
# confirmed_data_latest.head(100)

,Country/Region,Lat,Long,Confirmed,Deaths,Recovered,Infected
0,Afghanistan,33.0000,65.0000,120,4,2,114
1,Albania,41.1533,20.1683,212,10,33,169
2,Algeria,28.0339,1.6596,511,31,31,449
3,Andorra,42.5063,1.5218,334,6,1,327
4,Angola,-11.2027,17.8739,7,2,0,5
...,...,...,...,...,...,...,...
168,Venezuela,6.4238,-66.5897,119,2,39,78
169,Vietnam,16.0000,108.0000,188,0,25,163
170,West Bank and Gaza,31.9522,35.2332,109,1,18,90
171,Zambia,-15.4167,28.2833,29,0,0,29


In [188]:
# extractDataFromCountry(merged_dataset,'India')
extractDataFromCountry(merged_dataset,'Canada')

,Province/State,Country/Region,Lat,Long,Confirmed,Deaths,Recovered,Infected


In [195]:
dataset_copy = merged_dataset.groupby(['Province/State','Country/Region','Lat','Long'])[CONFIRMED,DEATH,RECOVERED,INFECTED].sum().reset_index()
stats = merged_dataset.groupby(['Country/Region'])[CONFIRMED,DEATH,RECOVERED,INFECTED].sum().reset_index()
# dataset_copy=dataset_copy.sort_values('Recovery Rate',ascending=False)
dataset_copy['size'] = dataset_copy['Confirmed'].pow(0.5)
# dataset_copy=dataset_copy.reset_index()
dataset_copy['HoverName'] = dataset_copy.apply(lambda x: (x[0]+' , '+x[1]) if x[0]!= 'NA' else (x[1]), axis=1)
dataset_copy['Outcome Cases']=dataset_copy['Confirmed']-dataset_copy['Infected']
dataset_copy['Death Rate']= (dataset_copy['Deaths']/dataset_copy['Outcome Cases'])*100
dataset_copy['Recovery Rate']= (dataset_copy['Recovered']/dataset_copy['Outcome Cases'])*100
dataset_copy=dataset_copy.sort_values('Recovery Rate',ascending=False)
stats['Outcome Cases']=stats['Confirmed']-stats['Infected']
stats['Death Rate']= (stats['Deaths']/stats['Outcome Cases'])*100
stats['Recovery Rate']= (stats['Recovered']/stats['Outcome Cases'])*100
stats=stats.sort_values('Recovery Rate',ascending=False)
stats=stats[stats['Country/Region']=='India']
stats

C:\Users\harsh\Anaconda3\envs\anomaly-detection\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

C:\Users\harsh\Anaconda3\envs\anomaly-detection\lib\site-packages\ipykernel_launcher.py:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,Country/Region,Confirmed,Deaths,Recovered,Infected,Outcome Cases,Death Rate,Recovery Rate
75,India,1024,27,95,902,122,22.131148,77.868852


In [190]:
stats.style.background_gradient(cmap='Pastel1_r')\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Coronavirus Stats")\
    .set_precision(2)

,Country/Region,Confirmed,Deaths,Recovered,Infected,Outcome Cases,Death Rate,Recovery Rate


In [191]:
projections= ['orthographic', 'natural earth', 'hammer']
for projection in projections:
    title = 'Spread in World over time | '+projection
    fig = px.scatter_geo(dataset_copy, lat='Lat', lon='Long',
                         color="size", size='size', hover_name='HoverName', 
                         hover_data=['Confirmed', 'Deaths', 'Recovered','Infected','Death Rate','Recovery Rate'],
                         projection=projection, 
                         title=title)
    fig.update(layout_coloraxis_showscale=False)
    fig.show()

In [192]:
def getHistogramDataByCountry(dataset,country):
    if country== "All":
        dataset=dataset.sum()
        historgram_data=dataset
    else:
        dataset=dataset.groupby(['Country/Region']).sum().reset_index()
        historgram_data=dataset[dataset['Country/Region']==country]
        historgram_data=historgram_data.T
    historgram_data=historgram_data.drop(historgram_data.index[[0,1,2,3]])
    historgram_data=historgram_data.reset_index()
    historgram_data.columns=['Date','Count']
    historgram_data=historgram_data.set_index('Date').diff()
    historgram_data=historgram_data.fillna(0)
    historgram_data=historgram_data.reset_index()
    return historgram_data

In [193]:
def getMergedDataForReport(COUNTRY):
    dataset=getHistogramDataByCountry(recovered_data,COUNTRY)
    dataset['type']=RECOVERED
    dataset1=getHistogramDataByCountry(confirmed_data,COUNTRY)
    dataset1['type']=CONFIRMED
    dataset2=getHistogramDataByCountry(death_data,COUNTRY)
    dataset2['type']=DEATH
    dataset=dataset.append(dataset1)
    dataset=dataset.append(dataset2)
    return dataset
def getConfirmedDataForReport(COUNTRY):
    dataset1=getHistogramDataByCountry(confirmed_data,COUNTRY)
    dataset1['type']=CONFIRMED
    return dataset1
def getDeathDataForReport(COUNTRY):
    dataset2=getHistogramDataByCountry(death_data,COUNTRY)
    dataset2['type']=DEATH
    return dataset2
def getRecoveredDataForReport(COUNTRY):
    dataset=getHistogramDataByCountry(recovered_data,COUNTRY)
    dataset['type']=RECOVERED
    return dataset


In [199]:
df=getMergedDataForReport('India')
rf=getRecoveredDataForReport('India')
ddf=getDeathDataForReport('India')
fig = px.line(df, x='Date', y='Count',color='type',range_y=[-1,df['Count'].max()+(10*df['Count'].max()/100)],range_x=[0,df.shape[0]/3+1])
fig.show()
fig = px.bar(df, x='Date', y='Count',color='type')
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=rf['Date'],y=rf['Count'],mode='lines+markers',
                    name='Recovered'))
fig.add_trace(go.Scatter(x=ddf['Date'],y=ddf['Count'],mode='lines+markers',
                    name='Death'))
# fig.update_layout(xaxis_type="log", yaxis_type="log")
fig.show()

# df=getMergedDataForReport('Canada')
# fig = px.line(df, x='Date', y='Count',color='type',range_y=[-1,df['Count'].max()+(10*df['Count'].max()/100)],range_x=[0,df.shape[0]/3+1])
# fig.show()
# fig = px.bar(df, x='Date', y='Count',color='type')
# fig.show()

# df=getMergedDataForReport('US')
# fig = px.line(df, x='Date', y='Count',color='type',range_y=[-1,df['Count'].max()+(10*df['Count'].max()/100)],range_x=[0,df.shape[0]/3+1])
# fig.show()
# fig = px.bar(df, x='Date', y='Count',color='type')
# fig.show()
